In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *

In [ ]:
'''
import os
import shutil 

alist=os.listdir('train')


for i in alist:
    if len(i)-4==5:
        shutil.move('train/'+i, 'train_proc/'+i[:-4]+'XX'+'.png')
    elif len(i)-4==6:
        shutil.move('train/'+i, 'train_proc/'+i[:-4]+'X'+'.png')
    elif len(i)-4==8:
        shutil.move('train/'+i, 'train_proc/'+i[:-5]+'.png')
    elif len(i)-4==7:
        shutil.move('train/'+i, 'train_proc/'+i[:-4]+'.png')
'''

In [5]:
import os
from pathlib import Path
DIR = 'captcha-version-2-images/samples'
path = Path(DIR)
os.listdir(path)[:10]

['226md.png',
 '22d5n.png',
 '2356g.png',
 '23mdg.png',
 '23n88.png',
 '243mm.png',
 '244e2.png',
 '245y5.png',
 '24f6w.png',
 '24pew.png']

In [ ]:
#labels = [[char for char in code.name[:-4]] for code in (path).glob('*.png')]
#labels = set([letter for label in labels for letter in label])
#print(len(labels), 'different labels were found')

In [ ]:
labels = [[char for char in code.name[:-4]] for code in (path).glob('*.png')]
labels = set([letter for label in labels for letter in label])
labels=list(labels)
labels.sort()
print(len(labels), 'different labels were found')

In [ ]:
encoding_dict = {l:e for e,l in enumerate(labels)}
decoding_dict = {e:l for l,e in encoding_dict.items()}

code_dimension = len(labels)
captcha_dimension = 7

In [ ]:
encoding_dict

In [ ]:
def to_onehot(filename):
    code = filename.name[:-4]
    onehot = np.zeros((code_dimension, captcha_dimension))
    for column, letter in enumerate(code):
        onehot[encoding_dict[letter], column] = 1
    return onehot.reshape(-1)

def to_idx(filename):
    code = filename.name[:-4]
    return np.array([encoding_dict[c] for c in code])#, dtype=torch.long)

def decode(onehot):
    onehot = onehot.reshape(code_dimension, captcha_dimension)
    idx = np.argmax(onehot, axis=0)
    return [decoding_dict[i.item()] for i in idx]

def label_accuracy(preds, actuals):
    pred = torch.unbind(preds)
    act = torch.unbind(actuals)
    
    valid = 0
    total = 0
    
    for left,right in zip(pred,act):
        total+=1
        p = decode(left)
        a = decode(right)
        if p==a: valid += 1

    return torch.tensor(valid/total).cuda()

def char_accuracy(n):
    def c_acc(preds, actuals):
        pred = torch.unbind(preds)
        act = torch.unbind(actuals)

        valid = 0
        total = 0

        for left,right in zip(pred,act):
            total+=1
            p = decode(left)
            a = decode(right)
            if p[n]==a[n]: valid += 1

        return torch.tensor(valid/total).cuda()
    return c_acc

In [ ]:
data = (ImageList.from_folder(path)
        .split_by_rand_pct(0.2,42)
        .label_from_func(to_onehot, label_cls = FloatList) #making it a regression instead of classification (because this gave better results)
        .transform(size=(77,247))
        .databunch(bs=64))
        #.normalize(imagenet_stats))

In [ ]:
#data.batchsize=64

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
path=Path('')

In [ ]:
learn = cnn_learner(data, models.resnet50, model_dir='models',pretrained=False,path=Path(''),
                    metrics=[label_accuracy, char_accuracy(0),char_accuracy(1),char_accuracy(2),char_accuracy(3),char_accuracy(4),char_accuracy(5),char_accuracy(6)],
                   ps=0.1)

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
# Next cell showed an error. https://forums.fast.ai/t/lesson-2-amazon-planet-cant-create-numpy-are-from-cuda-tensor/28380/7

# Got away with it by : 
# I went to the fastai code in the last file mentioned in the error(tensor.py) and replaced self.numpy() with self.cpu().numpy(). Your calls will automatically access this change.

# /home/sarwari/anaconda/envs/fastai/lib/python3.6/site-packages/torch/tensor.py This file was changed.

'''Changed 
def __array__(self, dtype=None):
        if dtype is None:
            return self.numpy()
        else:
            return self.numpy().astype(dtype, copy=False)
            
To

def __array__(self, dtype=None):
        if dtype is None:
            return self.cpu().numpy()
        else:
            return self.cpu().numpy().astype(dtype, copy=False)
            
'''

In [ ]:
#test
lr=5e-3
learn.fit_one_cycle(70,max_lr=lr,wd=0.01)

In [ ]:
path

In [ ]:
#learn.save('stage-1',return_path=True)

In [ ]:
learn.load('stage-1');

In [ ]:
img = open_image('test/ablger.png')
img

In [ ]:
defaults.device = torch.device('cpu')
pred_class,pred_idx,outputs = learn.predict(img)

In [ ]:
decode(pred_idx)

In [ ]:
img = open_image('test/caning.png')
img

In [ ]:
defaults.device = torch.device('cpu')
pred_class,pred_idx,outputs = learn.predict(img)

In [ ]:
decode(pred_idx)